In [1]:
! pip install sentence-splitter

     |████████████████████████████████| 51kB 7.9MB/s 


In [2]:
! pip install transformers

     |████████████████████████████████| 2.3MB 31.8MB/s 
     |████████████████████████████████| 3.3MB 42.8MB/s 
     |████████████████████████████████| 901kB 46.4MB/s 


In [3]:
! pip install SentencePiece

     |████████████████████████████████| 1.2MB 27.8MB/s 


In [4]:
import torch
from transformers import PegasusForConditionalGeneration, PegasusTokenizer

model_name = 'tuner007/pegasus_paraphrase'
torch_device = 'cuda' if torch.cuda.is_available() else 'cpu'
tokenizer = PegasusTokenizer.from_pretrained(model_name)
model = PegasusForConditionalGeneration.from_pretrained(model_name).to(torch_device)

def get_response(input_text,num_return_sequences):
  batch = tokenizer.prepare_seq2seq_batch([input_text],truncation=True,padding='longest',max_length=60, return_tensors="pt").to(torch_device)
  translated = model.generate(**batch,max_length=60,num_beams=10, num_return_sequences=num_return_sequences, temperature=1.5)
  tgt_text = tokenizer.batch_decode(translated, skip_special_tokens=True)
  return tgt_text

---

In [5]:
text = "When you get to the principal's office, write down the total number of kids on the bus."

In [6]:
get_response(text, 5)

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:3260: FutureWarning: `prepare_seq2seq_batch` is deprecated and will be removed in version 5 of 🤗 Transformers. Use the regular `__call__` method to prepare your inputs and the tokenizer under the `with_target_tokenizer` context manager to prepare your targets. See the documentation of your specific tokenizer for more details
  FutureWarning,


["Write down the number of kids on the bus when you get to the principal's office.",
 "Write down the total number of kids on the bus when you get to the principal's office.",
 "Write down the number of kids on the bus in the principal's office.",
 "Write down the number of kids on the bus in the principal's office when you get there.",
 "Write down the number of children on the bus in the principal's office."]

In [7]:
get_response(text, 1)

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:3260: FutureWarning: `prepare_seq2seq_batch` is deprecated and will be removed in version 5 of 🤗 Transformers. Use the regular `__call__` method to prepare your inputs and the tokenizer under the `with_target_tokenizer` context manager to prepare your targets. See the documentation of your specific tokenizer for more details
  FutureWarning,


["Write down the number of kids on the bus when you get to the principal's office."]

In [8]:
# Paragraph of text
context = "When you get to the principal's office, write down the total number of kids on the bus. This is important because we have to account for every kid on the bus. So, this is part of your daily routine now."
print(context)

When you get to the principal's office, write down the total number of kids on the bus. This is important because we have to account for every kid on the bus. So, this is part of your daily routine now.


In [9]:
# Takes the input paragraph and splits it into a list of sentences
from sentence_splitter import SentenceSplitter, split_text_into_sentences

splitter = SentenceSplitter(language='en')

sentence_list = splitter.split(context)
sentence_list

["When you get to the principal's office, write down the total number of kids on the bus.",
 'This is important because we have to account for every kid on the bus.',
 'So, this is part of your daily routine now.']

In [10]:
# Do a for loop to iterate through the list of sentences and paraphrase each sentence in the iteration
paraphrase = []

for i in sentence_list:
  a = get_response(i,1)
  paraphrase.append(a)

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:3260: FutureWarning: `prepare_seq2seq_batch` is deprecated and will be removed in version 5 of 🤗 Transformers. Use the regular `__call__` method to prepare your inputs and the tokenizer under the `with_target_tokenizer` context manager to prepare your targets. See the documentation of your specific tokenizer for more details
  FutureWarning,


In [11]:
# This is the paraphrased text
paraphrase

[["Write down the number of kids on the bus when you get to the principal's office."],
 ['We have to account for every kid on the bus.'],
 ['This is part of your daily routine.']]

In [12]:
paraphrase2 = [' '.join(x) for x in paraphrase]
paraphrase2

["Write down the number of kids on the bus when you get to the principal's office.",
 'We have to account for every kid on the bus.',
 'This is part of your daily routine.']

In [13]:
# Combines the above list into a paragraph
paraphrase3 = [' '.join(x for x in paraphrase2) ]
paraphrased_text = str(paraphrase3).strip('[]').strip("'")
paraphrased_text

'"Write down the number of kids on the bus when you get to the principal\'s office. We have to account for every kid on the bus. This is part of your daily routine."'

In [16]:
# Comparison of the original (context variable) and the paraphrased version (paraphrase3 variable)

print(context)
print('-' * 200)
print(paraphrased_text)

When you get to the principal's office, write down the total number of kids on the bus. This is important because we have to account for every kid on the bus. So, this is part of your daily routine now.
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
"Write down the number of kids on the bus when you get to the principal's office. We have to account for every kid on the bus. This is part of your daily routine."
